## Installs

In [1]:
!pip install langchain langchain_community langchain-openai langchainhub chromadb tiktoken

  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.meta

In [9]:
! pip install langchain-text-splitters -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [10]:
import os
import time
import json
import pprint
import pandas as pd

import langchain
print("langchain.__version__ ", langchain.__version__)

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

langchain.__version__  1.1.0


## Warm up & Config

In [12]:
raw_docs_base_dir = '../data/processed/p_jsons'

In [28]:
## check the len of each doc
all_len = []
all_char_len = []
for item in os.listdir(raw_docs_base_dir):
    with open(os.path.join(raw_docs_base_dir, item), 'r') as f:
        data = json.load(f)
        all_len.append(len(' '.join(data['doc_judgement']).split()))
        all_char_len.append(len(' '.join(data['doc_judgement'])))

pprint.pprint(pd.Series(all_len).describe())
pprint.pprint(pd.Series(all_char_len).describe())

print('Percent len > 7k: ', (len([item for item in all_len if item > 7000])/len(all_len)) * 100)

count      102.000000
mean      6559.382353
std       8863.852452
min        209.000000
25%       2441.750000
50%       3748.500000
75%       6385.750000
max      49445.000000
dtype: float64
count       102.000000
mean      43190.519608
std       58908.064283
min        1435.000000
25%       15841.750000
50%       23687.500000
75%       41734.750000
max      352851.000000
dtype: float64
Percent len > 7k:  22.54901960784314


## Indexing

In [ ]:
def custom_chunker(text: str):

    # based on len of doc, we can set different chunk size
    num_chars = len(text)

    if num_chars < 3000:
        return [text]
    
    elif num_chars > 3000 and num_chars < 12000:
        splitter = RecursiveCharacterTextSplitter(chunk_size=3000, 
                                                  chunk_overlap=300, 
                                                  separators=["\n\n", "\n", ".", " "])
        return splitter.split_text(text)
        
    else:
        coarse_splitter = RecursiveCharacterTextSplitter(chunk_size=9000,
                                                         chunk_overlap=900,
                                                         separators=["\n\n", "\n", ".", " "])
        coarse_chunks = coarse_splitter.split_text(text)
        fine_splitter = RecursiveCharacterTextSplitter(chunk_size=3000,
                                                       chunk_overlap=300,
                                                       separators=["\n\n", "\n", ".", " "])
        final_chunks = []
        for coarse_chunk in coarse_chunks:
            fine_chunks = fine_splitter.split_text(coarse_chunk)
            final_chunks.extend(fine_chunks)

        return final_chunks

## Retrieval

## Generation